In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
import seaborn as sns
from shapely.geometry import Point, LineString, MultiPoint
import numpy as np
import contextily as ctx
import requests
from io import StringIO, BytesIO
import json
import datetime as dt
from ast import literal_eval
from shapely.wkt import loads
import plotly.express as px
from dotenv import load_dotenv, find_dotenv

# intro
This notebook is quite a mess, so I'll explain: 
I use the ```nearbysearch``` [Link](https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests) to get all the ```bars``` and ```restaurants``` business hours. 

Then we fetch the unique id ```reference``` from the list of businesses and run it through the ```place_details``` API [Link](https://developers.google.com/maps/documentation/places/web-service/details)

From there we extract all the ```open``` (time-)elements and ```close``` (time-)elements and stack them in a dataframe divided by days of the week (0-6). 

In the end we merge the findings with an empty time series of 2018 with an 'hourly' sequence. 


In [2]:
# API KEY
load_dotenv(find_dotenv())
GOOGLE = os.environ.get("GOOGLE")

In [307]:
# first find all bars

url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {            
            'location':'45.05917,7.67899', #sensor
            'radius':'200',
            'type':'restaurant',
            'key':GOOGLE,
            'next_page_token':'Aap_uED24ODLIlOhPdAHG7xFrCg_OrsQ_jAruvTm3QSG4Qbnp5Q85Aa4K7ar-QgnGI7Xnl1epc9YIEj17piMfVpFUxQysBwi8XTzdWbtl6IBGKTKQwV_kxhaAUWr8JG6XVo-BVKHd8NJUwiTP-_uQvkKxc5vLZ4-v6T8ZBuS42zw5DE1L2KgNPCbm86EsPhPYOj8L1MXTRdEm_GhmQSdOt8nDxG4gKkbxiXvmHNTmuBLavqN-VrbpkRBBoVZz_t2P53_ShPgndMEwlt55EYlZHCYK2gHymy9WJjMjKn3VzS6CfcTQJ-TjgsxsrRjSqNXV4T5i2qusSJ__gsam11RBY8XRADB31i-ec_wYCh1529gNKKy9tdQbidVaQjAI72wQ-7yzTZXGzxpz8ob_DHkdVdyJLxijWoHqsXY7oQM-W3Db0u08SHwaooMyb3Da9Ij'
         } 

r = requests.get(url, params=params)
r.json()

{'html_attributions': [],
 'next_page_token': 'Aap_uEA8PsE_-RghVeOli5QzWZxVRiog5rpNtkMgKzXTuL1SAqIs9gOY1gbtR4oVHYxch21_qQNhT8DxxcnNCuN02eWNuaFkKxAFQOPUwkwbfjDNQd4jafTLYhyLayvKZ8t96Tj6g5gqCdVsRrallDToKsh3pGn9ixmVQ0-xhAOHPs2YFNU3EEgGkYnfMqYhzHML5xcJAgru83iMlsVLjVdlHufq1PleGB9cVjUPPI_ESxy6lz6L0YS5XNfufAa6R_pB0lz7N3HCd9SSmX8deqUiB-u1fVkZDrj7jIBEhjn3ajT6dzflhjaVVoL_kkxNCb9sWOZ_BW_lMtk70tK_-dcIyWZEfDLvnqCkf33B9owPQy-WS4nF7shYrL0nq2cZx-ftCPhU1Uzwy8CJWGuXdeleA8m2-AJ4PFpLrug4WAVhBXDbBJ41CEIlwxpvcxSF',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 45.0672255, 'lng': 7.6750973},
    'viewport': {'northeast': {'lat': 45.0687506302915,
      'lng': 7.676562230291502},
     'southwest': {'lat': 45.0660526697085, 'lng': 7.673864269708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': "Artua'&Solferino-Mezza Pensione-Parcheggio",
   'opening_hours': {'open_now': True},
   'photos': [{'height': 1536,
     'ht

In [298]:
results = r.json().get('results')


In [300]:
results2 = r.json().get('results')

In [302]:
results3 = r.json().get('results')

In [304]:
results4 = r.json().get('results')

In [306]:
results5 = r.json().get('results')

In [308]:
results6 = r.json().get('results')

In [256]:
bars = results + results2 + results3 + results4 + results5 + results6

In [309]:
restaurants = results + results2 + results3 + results4 + results5 + results6
len(restaurants)

120

In [310]:
# get specific opening hrs from fetched bars/restaurants
url = 'https://maps.googleapis.com/maps/api/place/details/json'
params = {
    'key':GOOGLE,
    'fields':'opening_hours'
         }
opening_hrs = []
for bar in restaurants:
    reference = bar.get('reference')
    params['place_id'] = reference
    r = requests.get(url, params=params)
    opening_hrs.append(r)


In [311]:
contents_hrs = [r.json() for r in opening_hrs]
periods = []
for x in contents_hrs:
    try:
        hr = x.get('result').get('opening_hours').get('periods')
        periods.append(hr)
    except:
        pass


In [312]:
# remove 24h open bars
new = [x for x in periods if len(x) > 1]

In [313]:
closing = []
for x in new:
    for i in x:
        _close = i.get('close')
        closing.append(_close)
opening = []
for x in new:
    for i in x:
        _open = i.get('open')
        opening.append(_open)


In [314]:
opening_times_rest = pd.DataFrame(opening)
closing_times_rest = pd.DataFrame(closing)
closing_times_rest['time'] = pd.to_datetime(closing_times_rest['time'], format='%H%M')
opening_times_rest['time'] = pd.to_datetime(opening_times_rest['time'], format='%H%M')
closing_times_rest['day'] = closing_times_rest.day.apply(lambda x: x-1 if x != 0 else 6)
opening_times_rest['day'] = opening_times_rest.day.apply(lambda x: x-1 if x != 0 else 6)

In [316]:
# create unique day_hr identifier
closing_times_rest['day_time'] = closing_times_rest.apply(lambda x: str(x.day) + '_' + str(x.time.hour), axis=1)
opening_times_rest['day_time'] = opening_times_rest.apply(lambda x: str(x.day) + '_' + str(x.time.hour), axis=1)



In [289]:
# put results in dataframe
opening_times = pd.DataFrame(opening)
closing_times = pd.DataFrame(closing)
closing_times['time'] = pd.to_datetime(closing_times['time'], format='%H%M')
opening_times['time'] = pd.to_datetime(opening_times['time'], format='%H%M')
closing_times['day'] = closing_times.day.apply(lambda x: x-1 if x != 0 else 6)
opening_times['day'] = opening_times.day.apply(lambda x: x-1 if x != 0 else 6)

In [290]:
# create unique day_hr identifier
closing_times['day_time'] = closing_times.apply(lambda x: str(x.day) + '_' + str(x.time.hour), axis=1)
opening_times['day_time'] = opening_times.apply(lambda x: str(x.day) + '_' + str(x.time.hour), axis=1)

In [320]:
closing_all = pd.concat([closing_times_rest, closing_times])
opening_all = pd.concat([opening_times_rest, opening_times])

In [321]:
# count all apperances of openings and closings per weekday
agg_close = closing_all.groupby('day_time').agg({'day':'count'}).rename(columns={'day':'count_close'})
agg_open = opening_all.groupby('day_time').agg({'day':'count'}).rename(columns={'day':'count_open'})
agg_joint = agg_close.join(agg_open, how='outer')

In [322]:
# init range 2018
range_2018 = pd.DataFrame(pd.date_range('2018-01-01', '2018-12-31', freq='h'), columns=['hour'])
range_2018['day_time'] =  range_2018.apply(lambda x: str(x.hour.weekday()) + '_' + str(x.hour.hour), axis=1)

In [323]:
# join both

opening_count_2018 = range_2018.merge(agg_joint, 
                                    on='day_time',
                                    how='left').drop(columns='day_time')

In [324]:
opening_times

,day,time,day_time
0,6,1900-01-01 06:30:00,6_6
1,0,1900-01-01 06:30:00,0_6
2,1,1900-01-01 06:30:00,1_6
3,2,1900-01-01 06:30:00,2_6
4,3,1900-01-01 06:30:00,3_6
...,...,...,...
829,1,1900-01-01 07:30:00,1_7
830,2,1900-01-01 07:30:00,2_7
831,3,1900-01-01 07:30:00,3_7
832,4,1900-01-01 07:30:00,4_7


In [330]:
opening_count_2018.sort_values(by='count_open')

,hour,count_close,count_open
6,2018-01-01 06:00:00,NaN,6.0
8422,2018-12-17 22:00:00,6.0,6.0
6849,2018-10-13 09:00:00,NaN,6.0
3395,2018-05-22 11:00:00,NaN,6.0
6851,2018-10-13 11:00:00,NaN,6.0
...,...,...,...
8732,2018-12-30 20:00:00,NaN,NaN
8733,2018-12-30 21:00:00,6.0,NaN
8734,2018-12-30 22:00:00,6.0,NaN
8735,2018-12-30 23:00:00,78.0,NaN


In [326]:
opening_count_2018.to_csv('raw_data/opening_count_2018.csv')

In [ ]:
# TODO: starting 0 on suday???
